# Data consistency

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import os

## Path

In [3]:
path = r'C:\Users\Hp 850 i5\Documents\Instacart Basket Analysis'

In [4]:
# Importing the product.csv data set
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [5]:
# Importing order_wrangled data set
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

# Data consistency checks

In [6]:
# Creating a dataframe
df_test = pd.DataFrame()

In [7]:
# Creating a mixed type column
df_test['mix'] = ['a', 'b', 1, True]

In [8]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [9]:
# checking the dataframe for mixed type column
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


C:\Users\Hp 850 i5\AppData\Local\Temp\ipykernel_10572\44074335.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)


In [10]:
#converting datatype from numeric to string
df_test['mix'] = df_test['mix'].astype('str')

## Finding missing values

In [11]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [12]:
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [13]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [14]:
# Checking the number of rows in the product dataframe
df_prods.shape

(49693, 5)

In [15]:
# Creating a new dataframe
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [16]:
df_prods_clean.shape

(49677, 5)

In [17]:
# Looking for duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [18]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [19]:
# creating new dataframe without duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [20]:
df_prods_clean_no_dups.shape

(49672, 5)

In [21]:
# Exporting the new dataframe as products_checked
df_prods.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))

In [22]:
# checking frequency
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


It looks from looking into our frequency table, especially the max and min sections, that days_since_prior_order has odd recordings. Although the max is three days and the min is zero, after double-checking the percentile range, it appears that the recording is odd. The 50th percentile for us is 7, over the maximum.

This is the situation for the amount of orders; despite the percentile ranges not matching this, we have a min and max of 1. For example, 25% is expressed as 5, which is greater than the maximum of 1.1.

## Checking for mixed data types

In [23]:
# Checking for mixed data types
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [24]:
 for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

C:\Users\Hp 850 i5\AppData\Local\Temp\ipykernel_10572\2412697239.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Hp 850 i5\AppData\Local\Temp\ipykernel_10572\2412697239.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Hp 850 i5\AppData\Local\Temp\ipykernel_10572\2412697239.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Hp 850 i5\AppData\Local\Temp\ipykernel_10572\2412697239.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Hp 

No mixed data type found in the df_ords dataframe

Hence nothing to fix

## Checking for missing values in df_ords dataframe.

In [25]:
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
hour_of_day_ordered            0
days_since_prior_order    206209
dtype: int64

In [26]:
# creating a new data frame for null values of column day_since_prior_order
df_ords_nan = df_ords[df_ords['days_since_prior_order'].isnull() == True]

In [27]:
df_ords_nan

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order
0,0,2539329,1,1,2,8,NaN
11,11,2168274,2,1,2,11,NaN
26,26,1374495,3,1,1,14,NaN
39,39,3343014,4,1,6,11,NaN
45,45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...,...
3420930,3420930,969311,206205,1,4,12,NaN
3420934,3420934,3189322,206206,1,3,18,NaN
3421002,3421002,2166133,206207,1,6,19,NaN
3421019,3421019,2227043,206208,1,1,15,NaN


In [29]:
# searching the number_of_orders column for missing values for each value in column.  
df_ords['order_number'].value_counts(dropna = False)

order_number
1      206209
2      206209
3      206209
4      206209
5      182223
        ...  
96       1592
97       1525
98       1471
99       1421
100      1374
Name: count, Length: 100, dtype: int64

The days_since_prior_order of 206209 has a sizable number of missing data. It is clear from each column that NaN and number of orders = 1 are related. The total number of user IDs (206209) is the same as the number of values we are missing. This indicates that the days_since_prior_order column, which pertains to the total number of orders, may include missing data for all user_id = 1. 
This makes logical because the customer making the initial purchase won't have any past orders, therefore that column will remain empty.

For question 6: The missing figure, by itself, tells us something about our customers and their orders, and it is consistent with the statistics of first-time orders. As a result, no action is necessary to remove the data because this makes the column relevant.

## Checking for duplicate values in df_ords data

In [30]:
df_ords_dups = df_ords[df_ords.duplicated()]

In [31]:
df_ords_dups

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order


No duplicates detected

No duplicates to address

## Exporting the Data frames df_ords and df_prods

In [33]:
df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.csv'))

In [34]:
df_prods.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))